# EV Clustering

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from IMPORT_DATAFRAME_JSON_HDF5 import *
import seaborn as sns
import matplotlib.pyplot as plt

## Import jsons/hdf5

Data time window: 02.03.2017 - 28.04.2017

In [3]:
data = import_trasient_from_file("jsons/")

100% (1370 of 1370) |#####################| Elapsed Time: 0:00:19 Time: 0:00:19


In [4]:
len(data)

1882

In [5]:
data = map_transients_to_PQ_data(data, "HDF5", 20, ['P'])

100% (1882 of 1882) |#####################| Elapsed Time: 0:05:54 Time: 0:05:54


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1882 entries, 1491026501.53 to 1490970806.68
Data columns (total 10 columns):
begin_index                1882 non-null int64
begin_timestamp_string     1882 non-null object
filtered_signal            1882 non-null object
phase_num                  1882 non-null int64
raw_signal_current         1882 non-null object
raw_signal_voltage         1882 non-null object
three_first_peaks          1882 non-null object
three_first_peaks_index    1882 non-null object
transient_rise_gradient    1882 non-null float64
P                          1882 non-null object
dtypes: float64(1), int64(2), object(7)
memory usage: 161.7+ KB


## Explore the data

This chapter should give a overview of the used dataset. Therefore step one is to study the header of the dataset. Here the column *'begin_timestamp_float'* can be used as a index, make our're events easier to find and to compare with other data sources. 

In [24]:
length = len(data)
print("Number of data points: {}".format(length))
data.head(1)

Number of data points: 1882


,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
begin_timestamp_float,,,,,,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",245806.997513,"[105.77301788330078, 104.60313415527344, 105.3...",0,105,0,0,0


In [25]:
transient_overview = pd.read_csv('transientlist.csv', sep=',', header=None)
transient_overview[0] = transient_overview[0].apply(convert_to_datetime)
len(transient_overview)

1370

In [26]:
transient_overview.sort_values(0).head(1)

,0,1,2,3
733,2017-01-04 08:01:41.528330,7681,[2],26.198


In [27]:
data.sort_values('begin_timestamp_string').head(1)

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
begin_timestamp_float,,,,,,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",245806.997513,"[105.77301788330078, 104.60313415527344, 105.3...",0,105,0,0,0


## Preprossing Data Seb

Combine transients data with power data.

In [18]:
data['charging_status']=0
data['P_mean']=0
data['P_delta'] = 0
data['P_Plugin']=0
data['P_Unplug'] = 0
for row in range(0, len(data)):
    # Getting the mean power before during and after the timestamp (-T, T)
    # Defining charging_status as 1 for avg(P)>300W else 0 and writing it to a new column
    P_mean = sum(data.iloc[row]['P']) / float(len(data.iloc[row]['P']))
    data.set_value(data.index[row],'P_mean', P_mean)
    if P_mean>300:
        data.set_value(data.index[row],'charging_status',int(1))
    else:
       data.set_value(data.index[row],'charging_status', int(0))
     # Defining if a new car is plugged in during the appearance of the transient
    P_mean_before = sum((data.iloc[row]['P'])[:10]) / float(len((data.iloc[row]['P'])[:10])) 
    P_mean_after = sum((data.iloc[row]['P'])[(len(data.iloc[row]['P'])-10):]) / float(len((data.iloc[row]['P'])[(len(data.iloc[row]['P'])-10):])) 
    P_delta = P_mean_after - P_mean_before
    data.set_value(data.index[row],'P_delta',P_delta)
    if P_delta > 2000:
        data.set_value(data.index[row],'P_Plugin',int(1))
    else:
        data.set_value(data.index[row],'P_Plugin',int(0))
    if P_delta < -2000:
        data.set_value(data.index[row],'P_Unplug',int(1))
    else:
        data.set_value(data.index[row],'P_Unplug',int(0))

In [21]:
data.head()

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
begin_timestamp_float,,,,,,,,,,,,,,,
1.491027e+09,701,01-04-2017_08:01:41_529732,"[7.9862e-06, 0.0259479208, -0.0389019368, 7.97...",2,"[0.6561279297, 0.7019042969, 0.7629394531, 0.7...","[289.647277832, 289.6991882324, 289.8029785156...","[26.198, 4.360, 2.076]","[754, 884, 1015]",2.458070e+05,"[105.77301788330078, 104.60313415527344, 105.3...",0,105,0,0,0
1.491027e+09,806,01-04-2017_08:14:53_521148,"[0.3013579845, 0.27552866940000004, 0.30145740...",2,"[0.732421875, 0.7171630859, 0.7476806641, 0.76...","[289.1282348633, 289.2579956055, 289.335845947...","[21.480, 3.592, 1.415]","[860, 1074, 1427]",1.951086e+05,"[105.6083755493164, 105.91771697998047, 105.88...",0,105,0,0,0
1.491073e+09,5717,01-04-2017_20:57:48_799036,"[-0.0624732189, -0.1910434365, 0.0152767645, -...",2,"[-1.2969970703, -1.2512207031, -1.2359619141, ...","[-292.2813720703, -292.3981628418, -292.618743...","[26.096, 4.280, 2.540]","[5769, 5892, 6032]",2.423178e+05,"[107.5003433227539, 107.1847915649414, 107.303...",0,106,-1,0,0
1.488462e+09,2750,02-03-2017_14:42:56_465712,"[-0.1300312473, 7.76288e-05, -0.0388483592, 0....",1,"[-2.6092529297000002, -2.6092529297000002, -2....","[0.1690305471, -0.1430258453, -0.3770681620000...","[58.447, 13.698, 33.141]","[2760, 2843, 3400]",-2.846197e+06,"[3704.182861328125, 3712.759521484375, 3715.73...",1,4850,3551,1,0
1.488462e+09,9697,02-03-2017_14:45:31_804547,"[0.142554611, 0.0336101092, 0.1024524048000000...",3,"[-31.0211181641, -31.1126708984, -31.127929687...","[-297.951385498, -297.7366638184, -297.6964111...","[40.280, 21.233, 2.845]","[9709, 9791, 9874]",-1.692315e+06,"[7219.86328125, 7217.51416015625, 7218.6166992...",1,8313,3503,1,0


## Preprossing Data

The next step is to Preprocessing the Data. In this present case it exist four different events to detect. At first transient cause from electric cars and the second on all from outside. In the following step all transients should be seperate in:

* ```events_from_outside```
* ```transients_1_phase```
* ```transients_2_phase```
* ```transients_3_phase```

This will be save into a pandas dataframe and export as a pkl file.

In [22]:
%%time
from collections import Counter
#documentation : https://docs.python.org/2/library/collections.html

# put all timestamps to a list
# and create a dict with Counter() to get a timestamp as key and a value how often we can find this timestamp
timestamps = data.index.tolist()
counter_timestamps = Counter()
for timestamp in timestamps:
    counter_timestamps[timestamp] += 1

# create a series from the counter object, that we can handle better the information
counter_timestamps_series = pd.Series(counter_timestamps)
data_point_num = len(counter_timestamps_series)
data_points_per_phase = counter_timestamps_series.value_counts()

# print all the result
print("Overview:")
print()
print("All events: {}".format(data_point_num))
print("Events with one phase: {}".format(data_points_per_phase[1]))
print("Events with two phases: {}".format(data_points_per_phase[2]))
print("Events with three phases: {}".format(data_points_per_phase[3]))

Overview:

All events: 1568
Events with one phase: 1305
Events with two phases: 212
Events with three phases: 51
CPU times: user 4.98 ms, sys: 2.5 ms, total: 7.48 ms
Wall time: 6.59 ms


In [23]:
# import a function that can seperate trasients by event types and extract some features
from SEPERATE_DF import *

In [24]:
%time events_from_outside, transients_1_phase, transients_2_phase, transients_3_phase = seperate_transients(data)

CPU times: user 10.5 s, sys: 97.3 ms, total: 10.6 s
Wall time: 10.7 s


In [25]:
# export df to csv files
events_from_outside.to_pickle("events_from_outside.pkl")
transients_1_phase.to_pickle("transients_1_phase.pkl")
transients_2_phase.to_pickle("transients_2_phase.pkl")
transients_3_phase.to_pickle("transients_3_phase.pkl")

In [26]:
len(events_from_outside)

1543

In [27]:
transients_1_phase = pd.read_pickle("transients_1_phase.pkl")

In [28]:
len(transients_1_phase)

252

In [29]:
filtered_signal_1_ph = transients_1_phase["filtered_signal"].apply(pd.Series).fillna(0)

In [30]:
filtered_signal_1_ph_split = filtered_signal_1_ph.iloc[:50,:].transpose()

In [31]:
filtered_signal_1_ph_split.columns = [i for i in range(filtered_signal_1_ph_split.shape[1])]

In [32]:
filtered_signal_1_ph_split.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.130031,0.142555,0.107352,-0.532786,-0.586819,0.051739,0.201216,-0.488330,-0.182076,0.160802,...,-0.238017,-0.191268,0.011405,-0.367459,-0.051920,-0.104165,-0.555143,0.164927,0.117020,0.183388
1,0.000078,0.033610,0.093924,-0.585368,-0.392881,0.155515,0.147579,-0.488329,-0.025976,0.174081,...,-0.236418,-0.087424,0.010599,-0.300272,0.090811,-0.158881,-0.582234,0.072177,0.013002,0.196368
2,-0.038848,0.102452,-0.040262,-0.324715,-0.509044,0.064660,0.187913,-0.515061,-0.051718,0.133697,...,-0.237904,-0.009572,-0.069056,-0.246520,0.038931,-0.131075,-0.607088,0.165044,0.078013,0.131366
3,0.039155,0.181336,-0.174478,-0.208262,-0.392948,-0.000396,0.268410,-0.421197,-0.090714,0.120354,...,-0.289961,0.003415,-0.029615,-0.286856,0.012961,0.055805,-0.543428,0.150190,-0.104020,0.014606
4,0.039402,0.035356,-0.161050,-0.051576,-0.483057,0.077473,0.241621,-0.461278,0.052395,-0.000767,...,-0.184094,0.029481,-0.042151,-0.179263,-0.012979,0.029879,-0.620049,0.113297,-0.013003,-0.024533


In [33]:
# rearrange voltage column-wise, not row-wise

master_df = pd.DataFrame(filtered_signal_1_ph_split[0])
master_df['id'] = 0

bar = progressbar.ProgressBar()

with progressbar.ProgressBar(max_value=len(filtered_signal_1_ph_split.columns)) as bar:
    for i in range(1,len(filtered_signal_1_ph_split.columns)): #len(df_raw_signal_voltage_t.columns)
        signal_df = pd.DataFrame(filtered_signal_1_ph_split[i])
        signal_df['id'] = i
        master_df = pd.DataFrame(np.vstack([master_df, signal_df]))
        bar.update(i)

100% (50 of 50) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00


In [34]:
master_df.head()

,0,1
0,-0.130031,0.0
1,0.000078,0.0
2,-0.038848,0.0
3,0.039155,0.0
4,0.039402,0.0


In [43]:
sns.set(style="ticks")

# Initialize a grid of plots with an Axes for each transient
grid = sns.FacetGrid(master_df, col=1, hue=1, col_wrap=2, size=10)

# Draw a line plot to show the trajectory of each random walk
grid.map(plt.plot, 0, ms=4)

In [44]:
len(transients_2_phase)

170

In [47]:
transients_3_phase

,begin_index,begin_timestamp_string,filtered_signal,phase_num,raw_signal_current,raw_signal_voltage,three_first_peaks,three_first_peaks_index,transient_rise_gradient,P,charging_status,P_mean,P_delta,P_Plugin,P_Unplug
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[0.18232499060000001, 0.1427274048, 0.18223194...",1,"[-1.9378662109, -2.0141601562, -2.0141601562, ...","[0.0390070491, -0.1430258453, -0.4550822377, -...","[22.142, 7.970, 18.781]","[4023, 4042, 4779]",1.535173e+06,"[193.2838592529297, 191.64263916015625, 194.01...",1,4914,3543,1,0
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[0.3145695329, 0.3665215075, 0.3665425181, 0.3...",2,"[20.5841064453, 20.6451416016, 20.6909179688, ...","[290.9189147949, 290.9967651367, 291.061645507...","[24.063, 7.688, 16.598]","[4023, 4107, 4781]",1.677861e+06,"[3659.302490234375, 3659.780029296875, 3660.46...",1,4914,3543,1,0
1.491395e+09,4016,05-04-2017_14:23:22_406877,"[-0.016060669, 0.06456168, 0.1584541202, 0.158...",3,"[-15.4571533203, -15.4113769531, -15.380859375...","[-294.582824707, -294.4351806641, -294.3949279...","[69.691, 21.316, 45.636]","[4024, 4052, 4782]",4.173877e+06,"[3698.833740234375, 3698.317138671875, 3701.89...",1,4914,3543,1,0
1.488965e+09,3508,08-03-2017_10:27:50_128345,"[0.15550541880000002, 0.15653982760000001, 0.2...",1,"[-1.953125, -1.953125, -1.9226074219, -1.89208...","[-0.013002350000000001, -0.2080375999, -0.5200...","[23.521, 9.473, 20.440]","[3516, 3535, 4332]",1.466010e+06,"[199.44705200195312, 189.89479064941406, 200.9...",1,4866,3477,1,0
1.488965e+09,3508,08-03-2017_10:27:50_128345,"[0.2236295193, 0.28847584130000004, 0.36641946...",2,"[20.2789306641, 20.3094482422, 20.3399658203, ...","[289.1542053223, 289.2450256348, 289.309906005...","[25.649, 9.155, 21.862]","[3516, 3561, 4331]",1.593177e+06,"[3603.539794921875, 3602.707275390625, 3602.30...",1,4866,3477,1,0
1.488965e+09,3508,08-03-2017_10:27:50_128345,"[-0.1230454743, -0.2709324956, -0.0693374872, ...",3,"[-15.7470703125, -15.686035156199999, -15.7165...","[-292.6770935059, -292.4489440918, -292.448944...","[73.768, 24.713, 60.850]","[3517, 3541, 4332]",4.090313e+06,"[3691.115234375, 3691.073486328125, 3691.33496...",1,4866,3477,1,0
1.489051e+09,5429,09-03-2017_10:21:22_186854,"[0.11703932700000001, -0.065030892, 0.06490481...",1,"[3.3569335938, 3.41796875, 3.4027099609, 3.356...","[-0.2990540564, 0.0390070491, 0.2470446527, 0....","[22.261, 5.339, 15.579]","[5438, 5457, 6043]",1.227302e+06,"[3624.978515625, 3619.1474609375, 3620.2145996...",0,126,3,1,0
1.489051e+09,5429,09-03-2017_10:21:22_186854,"[0.0422243389, -0.1264481316, -0.0356074469, -...",2,"[-1.1291503906, -1.1138916016, -1.0986328125, ...","[-292.5798339844, -292.5538635254, -292.696594...","[95.814, 26.040, 65.944]","[5437, 5461, 6042]",5.796642e+06,"[111.72794342041016, 112.19425201416016, 111.9...",0,126,3,1,0
1.489051e+09,5429,09-03-2017_10:21:22_186854,"[-0.0293289288, -0.2037554952, -0.082980824000...",3,"[-0.9307861328, -0.9918212891, -1.0375976562, ...","[293.2810058594, 293.2273254395, 293.052856445...","[41.611, 10.809, 31.085]","[5437, 5454, 6040]",2.556301e+06,"[124.93058776855469, 124.82858276367188, 125.3...",0,126,3,1,0
1.489121e+09,6372,10-03-2017_05:41:47_261420,"[-0.0654577538, 0.0004888828, -0.0132960640000...",1,"[2.2277832031, 2.2735595702999998, 2.288818359...","[-0.0780140981, 0.1300234944, 0.2860516906, 0....","[22.418, 13.847, 7.541]","[6382, 6399, 6425]",1.109819e+06,"[186.13565063476562, 185.5493927001953, 180.47...",0,132,4,1,0


In [45]:
len(transients_3_phase)

63

### Create Feature Dataframe from exist features

The Problem with time series is, that they are high dimensional. Therefore it's important to extract features from this time series to reduce the dimensions of the data. So the next step is to preprocessing our data, that we can extract features from this Dataframe. We will create a Dataframe for the current/voltage signals.

## Feature for one phase events

In [ ]:
from CREATE_DF import *

In [ ]:
%time feature_df_with_nan_1 = create_feature_df(transients_1_phase)

In [ ]:
# sort df: The code put all NaN values in the last columns of the df
feature_df_with_nan_sort_1 = feature_df_with_nan_1.apply(squeeze_nan, axis=1)
feature_df_with_nan_sort_1.reset_index(level=0, inplace=True)

# get columns name an put them into a list
original_columns = feature_df_with_nan_sort_1.columns.tolist()
# Create new column names
new_columns = original_columns[:4] + original_columns[10:13] + original_columns[19:20] + original_columns[22:23]
# drop all columns with NaN
feature_df_1_phase = feature_df_with_nan_sort_1.dropna(axis=1)
feature_df_1_phase = feature_df_1_phase.apply(pd.to_numeric)
feature_df_1_phase.columns = new_columns

In [ ]:
feature_df_1_phase.to_csv("feature_df_1_phase.csv", columns = new_columns, index=None)

## Feature for two phase events

In [ ]:
%time feature_df_with_nan_2 = create_feature_df(transients_2_phase)

# sort df: The code put all NaN values in the last columns of the df
feature_df_with_nan_sort_2 = feature_df_with_nan_2.apply(squeeze_nan, axis=1)
feature_df_with_nan_sort_2.reset_index(level=0, inplace=True)
# get columns name an put them into a list
original_columns = feature_df_with_nan_sort_2.columns.tolist()
# Create new column names
new_columns = original_columns[:7] + original_columns[10:16] + original_columns[19:21] + original_columns[22:24]
# drop all columns with NaN
feature_df_2_phase = feature_df_with_nan_sort_2.dropna(axis=1)
feature_df_2_phase = feature_df_2_phase.apply(pd.to_numeric)
feature_df_2_phase.columns = new_columns

In [ ]:
feature_df_2_phase.to_csv("feature_df_2_phase.csv", columns = new_columns, index=None)

## Feature for three phase events

In [ ]:
%time feature_df_3_phase = create_feature_df(transients_3_phase)

In [ ]:
feature_df_3_phase.reset_index(level=0, inplace=True)

In [ ]:
feature_df_3_phase.head()

In [ ]:
feature_df_3_phase.to_csv("feature_df_3_phase.csv", index=None)

In [ ]:
test = pd.read_csv("feature_df_3_phase.csv")

In [ ]:
test.head()

# Spielen

In [ ]:
Z = linkage(feature_df, 'ward')

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(feature_df))
c

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=7,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,
    max_d=16,
)
plt.show()

In [ ]:
# create trasient data frame
raw_signal_current = data["raw_signal_current"]
raw_signal_voltage = data["raw_signal_voltage"]

# create dataframes
# on row contains on time series
# column is time or dimension
df_raw_signal_current = pd.DataFrame([x for x in raw_signal_current])
df_raw_signal_voltage = pd.DataFrame([x for x in raw_signal_voltage])

In [ ]:
df_raw_signal_current.head(1)

In [ ]:
df_raw_signal_voltage.head(1)

### Clustering

### Extract Features Voltage (mit tsfresh)

Quelle Bild:
https://tsfresh.readthedocs.io/en/latest/_images/feature_extraction_process_20160815_mc_1.png

In [ ]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import FeatureExtractionSettings

extraction_settings = FeatureExtractionSettings()
extraction_settings.IMPUTE = impute    # Fill in Infs and NaNs

# transpose since tsfresh reads times series data column-wise, not row-wise
df_raw_signal_voltage_t = df_raw_signal_voltage.copy().transpose()
df_raw_signal_voltage_t_cut = df_raw_signal_voltage_t[:100]

print(df_raw_signal_voltage_t_cut.shape)
len(df_raw_signal_voltage_t_cut.columns)

In [ ]:
# rearrange voltage column-wise, not row-wise

master_df = pd.DataFrame(df_raw_signal_voltage_t_cut[0])
master_df['id'] = 0

bar = progressbar.ProgressBar()

with progressbar.ProgressBar(max_value=len(df_raw_signal_voltage_t_cut.columns)) as bar:
    for i in range(1,500): #len(df_raw_signal_voltage_t.columns)
        signal_df = pd.DataFrame(df_raw_signal_voltage_t_cut[i])
        signal_df['id'] = i
        master_df = pd.DataFrame(np.vstack([master_df, signal_df]))
        bar.update(i)

In [ ]:
master_df.shape

In [ ]:
new_master_df = master_df.dropna(how='any')
new_master_df.shape

In [ ]:
%time X = extract_features(new_master_df, column_id=1, feature_extraction_settings=extraction_settings)